In [15]:
!pip install scikit-learn

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

In [2]:
np.random.seed(0)

In [3]:
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples = n_samples, factor = .5, noise =.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples,random_state = 8)
no_structure = np.random.rand(n_samples, 2), None

In [4]:
# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6],[-0.4,0.8]] #?
X_aniso = np.dot(X, transformation) #?
aniso = (X_aniso, y)

In [5]:
# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state)

In [20]:
#======
# set up cluster parameters
plt.figure(figsize=(9*2 + 3, 12.5))
plt.subplots_adjust(left =.02, right = .98, bottom= .001, top =.96, wspace =.05, hspace = .01)
plt_num = 1

default_base = {'quatile':3,
                'eps':.3,
                'damping':.9,
               'preference':-200,
               'n_neighbors':10,
               'n_clusters':3}

datasets = [
    (noisy_circles, {'damping':.77, 'performance':-240,
                    'quantile':.2, 'n_clusters':2}), 
    (noisy_moons, {'damping':.75 , 'performance': -220, 'n_clusters': 2}),
    (varied,{'eps':.18, 'n_neighbors':2}),
    (aniso, {'eps':.15, 'n_neighors':2}),
    (blobs,{}),
    (no_structure, {})
]

<Figure size 1512x900 with 0 Axes>

In [21]:
# for i_dataset, (dataset, algo_params) in enumerate(datasets):
#     # update parameters with dataset-specific values
#     params = default_base.copy()
#     params.update(algo_params)
    
#     X, y = dataset
#     X = StandardScaler().fit_transform(X)

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)
    
    # estimate bandwidth for mean shift
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5*(connectivity + connectivity.T)
    
    
    #create cluster objects
    ms = cluster.MeanShift(bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters)'])
    ward = cluster.AgglomerativeClustering(
        n_clusters = params['n_clusters'], linkage = 'ward', connectivity = connectivity)
    spectral = cluster.SpectralBiclustering(
        n_clusters=params['n_clusters'],eigen_colver='arpack', affinity = 'nearest_neighbors')
    dbscan = cluster.DBSCAN(eps=params['eps'])
    affinity_propagation = cluster.AgglomerativeClustering(
        linkage = "average". affinity = 'cityblock',
        n_cluster = params['n_clusters'], connectivity = connectivity)
    